# BALLBOT SYSTEM: EQUATIONS OF MOTION AND LINEARIZATION
# Derivation using Lagrangian mechanics and linearization for control design

In [2]:
import sympy as sp
from sympy import symbols, Matrix, cos, sin, diff, simplify, latex
from sympy.physics.mechanics import dynamicsymbols, Lagrangian
import numpy as np


# 1. DEFINE SYMBOLIC PARAMETERS

In [3]:
# System parameters (in SI units, from MATLAB)
g, M_r, m_b, d, h, r, l = symbols('g M_r m_b d h r l', real=True, positive=True)
I_r, I_b = symbols('I_r I_b', real=True, positive=True)

# Time-dependent generalized coordinates
t = sp.Symbol('t')
x = dynamicsymbols('x', real=True)           # ball horizontal position
theta = dynamicsymbols('theta', real=True)   # body pitch angle

print(f"Generalized coordinates: q = [x(t), θ(t)]ᵀ")
print(f"System parameters: M_r, m_b, d, h, r, l, I_r, I_b, g")
print(f"Coupling parameter: l = h + r")

Generalized coordinates: q = [x(t), θ(t)]ᵀ
System parameters: M_r, m_b, d, h, r, l, I_r, I_b, g
Coupling parameter: l = h + r


# 2. KINETIC ENERGY

In [5]:
# From linearized kinematics: ball center velocity involves both ẋ and θ̇
# Effective kinetic energy (after simplification):
T_ball_trans = sp.Rational(1, 2) * (I_b / r**2 + m_b) * x.diff(t)**2
T_ball_rot_body = sp.Rational(1, 2) * (I_r + m_b * l**2) * theta.diff(t)**2
T_coupling = m_b * l * x.diff(t) * theta.diff(t)

T = T_ball_trans + T_ball_rot_body + T_coupling

print(f"Translational KE (ball + rolling inertia):")
print(f"  T_trans = (1/2) * (I_b/r² + m_b) * ẋ²")
print(f"\nRotational KE (body + ball about axle):")
print(f"  T_rot = (1/2) * (I_r + m_b*l²) * θ̇²")
print(f"\nCoupling term:")
print(f"  T_coupling = m_b * l * ẋ * θ̇")
print(f"\nTotal kinetic energy:")
print(f"  T = {T}")

Translational KE (ball + rolling inertia):
  T_trans = (1/2) * (I_b/r² + m_b) * ẋ²

Rotational KE (body + ball about axle):
  T_rot = (1/2) * (I_r + m_b*l²) * θ̇²

Coupling term:
  T_coupling = m_b * l * ẋ * θ̇

Total kinetic energy:
  T = l*m_b*Derivative(theta(t), t)*Derivative(x(t), t) + (I_r/2 + l**2*m_b/2)*Derivative(theta(t), t)**2 + (I_b/(2*r**2) + m_b/2)*Derivative(x(t), t)**2


# 3. POTENTIAL ENERGY

In [6]:
# Gravity acting on body COM and ball center
U_body = M_r * g * d * cos(theta)
U_ball = m_b * g * ((h + r) * cos(theta) - x * sin(theta))
U = U_body + U_ball

print(f"Potential energy (body COM):")
print(f"  U_body = M_r * g * d * cos(θ)")
print(f"\nPotential energy (ball center):")
print(f"  U_ball = m_b * g * [(h+r)*cos(θ) - x*sin(θ)]")
print(f"\nTotal potential energy:")
print(f"  U = U_body + U_ball")

Potential energy (body COM):
  U_body = M_r * g * d * cos(θ)

Potential energy (ball center):
  U_ball = m_b * g * [(h+r)*cos(θ) - x*sin(θ)]

Total potential energy:
  U = U_body + U_ball


# 4. LAGRANGIAN

In [7]:
L = T - U
print(f"Lagrangian L = T - U (symbolic expression computed)")

Lagrangian L = T - U (symbolic expression computed)


# 5. NONLINEAR EQUATIONS OF MOTION (Euler-Lagrange)

In [8]:
print("\n[5] NONLINEAR EQUATIONS OF MOTION (Euler-Lagrange)")
print("-" * 80)
print("Applying: d/dt(∂L/∂q̇) - ∂L/∂q = Q")

# Apply Euler-Lagrange for x
dL_dx_dot = diff(L, x.diff(t))
d_dt_dL_dx_dot = diff(dL_dx_dot, t)
dL_dx = diff(L, x)
EOM_x = d_dt_dL_dx_dot - dL_dx

# Apply Euler-Lagrange for theta
dL_dtheta_dot = diff(L, theta.diff(t))
d_dt_dL_dtheta_dot = diff(dL_dtheta_dot, t)
dL_dtheta = diff(L, theta)
EOM_theta = d_dt_dL_dtheta_dot - dL_dtheta

print(f"\nNonlinear EOM for x:   (after simplification)")
print(f"  {simplify(EOM_x)} = Q_x")
print(f"\nNonlinear EOM for θ:   (after simplification)")
print(f"  {simplify(EOM_theta)} = Q_θ")


[5] NONLINEAR EQUATIONS OF MOTION (Euler-Lagrange)
--------------------------------------------------------------------------------
Applying: d/dt(∂L/∂q̇) - ∂L/∂q = Q

Nonlinear EOM for x:   (after simplification)
  (m_b*r**2*(-g*sin(theta(t)) + l*Derivative(theta(t), (t, 2))) + (I_b + m_b*r**2)*Derivative(x(t), (t, 2)))/r**2 = Q_x

Nonlinear EOM for θ:   (after simplification)
  -M_r*d*g*sin(theta(t)) - g*m_b*((h + r)*sin(theta(t)) + x(t)*cos(theta(t))) + l*m_b*Derivative(x(t), (t, 2)) + (I_r + l**2*m_b)*Derivative(theta(t), (t, 2)) = Q_θ


# 6. LINEARIZATION (small-angle approximation)

In [9]:
print("\n[6] LINEARIZATION ABOUT UPRIGHT EQUILIBRIUM")
print("-" * 80)
print("Equilibrium: x = 0, θ = 0, all velocities = 0")
print("Approximations: sin(θ) ≈ θ,  cos(θ) ≈ 1")
print("Dropping: products of small quantities (e.g., x*θ̇², x*θ*accel, etc.)")

# Substitute small-angle approximations
EOM_x_lin = EOM_x.subs([(sin(theta), theta), (cos(theta), 1)])
EOM_theta_lin = EOM_theta.subs([(sin(theta), theta), (cos(theta), 1)])

# Remove nonlinear terms (products of small quantities)
# For simplicity, we identify the coefficient of each state and acceleration
EOM_x_lin = simplify(EOM_x_lin)
EOM_theta_lin = simplify(EOM_theta_lin)

print(f"\nLinearized EOM for x:   {EOM_x_lin} = Q_x")
print(f"\nLinearized EOM for θ:   {EOM_theta_lin} = Q_θ")


[6] LINEARIZATION ABOUT UPRIGHT EQUILIBRIUM
--------------------------------------------------------------------------------
Equilibrium: x = 0, θ = 0, all velocities = 0
Approximations: sin(θ) ≈ θ,  cos(θ) ≈ 1
Dropping: products of small quantities (e.g., x*θ̇², x*θ*accel, etc.)

Linearized EOM for x:   (m_b*r**2*(-g*theta(t) + l*Derivative(theta(t), (t, 2))) + (I_b + m_b*r**2)*Derivative(x(t), (t, 2)))/r**2 = Q_x

Linearized EOM for θ:   -M_r*d*g*theta(t) - g*m_b*((h + r)*theta(t) + x(t)) + l*m_b*Derivative(x(t), (t, 2)) + (I_r + l**2*m_b)*Derivative(theta(t), (t, 2)) = Q_θ


# 7. EXTRACT M1, M2, M3 MATRICES

In [10]:
print("\n[7] MATRIX FORM: M1*q̈ = M2*q + M3*u")
print("-" * 80)

# Inertia matrix (M1) - coefficients of accelerations
M1 = Matrix([
    [I_b/r**2 + m_b, m_b*l],
    [m_b*l, I_r + m_b*l**2]
])

# Gravity matrix (M2) - coefficients of positions
M2 = Matrix([
    [0, m_b*g],
    [m_b*g, (M_r*d + m_b*l)*g]
])

# Input matrix (M3) - actuator torque effect
M3 = Matrix([
    [-m_b*l],
    [-(I_r + m_b*l**2)]
])

print(f"\nInertia Matrix M1 (2×2):")
print(f"  M1 = [[  (I_b/r² + m_b),     m_b*l      ],")
print(f"        [     m_b*l,        I_r + m_b*l²  ]]")
print(f"\nGravity/Stiffness Matrix M2 (2×2):")
print(f"  M2 = [[        0,              m_b*g          ],")
print(f"        [     m_b*g,      (M_r*d + m_b*l)*g  ]]")
print(f"\nInput Matrix M3 (2×1):")
print(f"  M3 = [[-m_b*l],")
print(f"        [-(I_r + m_b*l²)]]")

print(f"\nLaTeX form of M1:")
print(f"  {latex(M1)}")
print(f"\nLaTeX form of M2:")
print(f"  {latex(M2)}")
print(f"\nLaTeX form of M3:")
print(f"  {latex(M3)}")


[7] MATRIX FORM: M1*q̈ = M2*q + M3*u
--------------------------------------------------------------------------------

Inertia Matrix M1 (2×2):
  M1 = [[  (I_b/r² + m_b),     m_b*l      ],
        [     m_b*l,        I_r + m_b*l²  ]]

Gravity/Stiffness Matrix M2 (2×2):
  M2 = [[        0,              m_b*g          ],
        [     m_b*g,      (M_r*d + m_b*l)*g  ]]

Input Matrix M3 (2×1):
  M3 = [[-m_b*l],
        [-(I_r + m_b*l²)]]

LaTeX form of M1:
  \left[\begin{matrix}\frac{I_{b}}{r^{2}} + m_{b} & l m_{b}\\l m_{b} & I_{r} + l^{2} m_{b}\end{matrix}\right]

LaTeX form of M2:
  \left[\begin{matrix}0 & g m_{b}\\g m_{b} & g \left(M_{r} d + l m_{b}\right)\end{matrix}\right]

LaTeX form of M3:
  \left[\begin{matrix}- l m_{b}\\- I_{r} - l^{2} m_{b}\end{matrix}\right]
